In [1]:
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import numpy as np
import random as rd

First need to create k Gaussians for a mixture

In [2]:
# The book suggested first running k means a few times and then using that as those as the Gaussians

iris = load_iris()
data = iris.data
target = iris.target

XFull = data[:, 1:3]
XTrain, XTest, yTrain, yTest = train_test_split(XFull, target)

yTrain = yTrain.reshape(-1,1)
yTest = yTest.reshape(-1,1)
XTrain.shape, yTrain.shape

((112, 2), (112, 1))

In [3]:
k = 3
maxIter = 5
kmeans = KMeans(n_clusters = k, max_iter = maxIter)
kmeans.fit(XTrain, yTrain)

pointsInEachCluster = [XTrain[kmeans.labels_ == i] for i in range(k)]
print(len(pointsInEachCluster[0]))
print(len(pointsInEachCluster[1]))
print(len(pointsInEachCluster[2]))

c:\Users\unova\anaconda3\envs\nb\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


52
38
22


In [4]:
centers = kmeans.cluster_centers_
means = []
covMats = []
for i in range(len(pointsInEachCluster)):
    points = pointsInEachCluster[i]
    means.append(centers[i])
    covMat = np.cov(points.T)
    covMats.append(covMat)

means[0], covMats[0]

(array([2.81346154, 4.49230769]),
 array([[0.09059955, 0.06598793],
        [0.06598793, 0.21562594]]))

In [5]:
prior = 1 / len(means)

In [29]:
dist = np.random.multivariate_normal(means[0], covMats[0])
dist

array([2.76359728, 4.80704301])

In [ ]:
# I need take the current values of means, covariance and priors and calculate the h_ti

# h_ti is the posterior probability of the ith data point belonging to the ith cluster for the t_th data point given the point
# and the previous values of the means, covariance and priors

# so just a normal distribution with the mean and covariance of the cluster